<a href="https://colab.research.google.com/github/aishwaryaprabhat/Advanced-RAG/blob/main/Advanced_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Download and Environment Preparation

In [1]:
# !echo -e "# Clone the repository\ngit clone https://github.com/AI-Maker-Space/DataRepository.git\n\n# Create the 'source_docs' directory in the current working directory\nmkdir source_docs\n\n# Unzip files directly into 'source_docs' directory without preserving internal folder structure\nunzip -j \"DataRepository/high-performance-rag/Camel Papers Test.zip\" -d source_docs\nunzip -j \"DataRepository/high-performance-rag/Camel Papers Train.zip\" -d source_docs\n\n# Delete the cloned repository\nrm -rf DataRepository" > download_dataset.sh
!bash download_dataset.sh

Cloning into 'DataRepository'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 47 (delta 12), reused 21 (delta 7), pack-reused 8
Receiving objects: 100% (47/47), 49.80 MiB | 35.93 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Archive:  DataRepository/high-performance-rag/Camel Papers Test.zip
  inflating: source_docs/Acute respiratory distress syndrome in an alpaca cria.pdf  
  inflating: source_docs/Alpaca liveweight variations and fiber production in Mediterranean range of Chile.pdf  
Archive:  DataRepository/high-performance-rag/Camel Papers Train.zip
  inflating: source_docs/Antibody response to the epsilon toxin ofClostridium perfringensfollowing vaccination of Lama glamacrias.pdf  
  inflating: source_docs/Comparative pigmentation of sheep, goats, and llamas what colors are possible through selection.pdf  
  inflating: source_docs/Conservative management of a ruptured.pdf  


In [127]:
# !pip install llama-index pypdf sentence_transformers typing_extensions==4.7.1 nest_asyncio -U -q
!pip install -r requirements.txt -U -q

In [21]:
import os
from google.colab import userdata
import nest_asyncio

nest_asyncio.apply()
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['HUGGINGFACE_API_TOKEN'] = userdata.get('HF_TOKEN')

In [25]:
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms import OpenAI
from llama_index import SimpleDirectoryReader

# Initialize an embedding model from Hugging Face using the "BAAI/bge-small-en" model.
embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

# Create an OpenAI GPT-3.5 model instance with no randomness in responses (temperature=0).
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

# Load data from a directory named 'source_docs' using SimpleDirectoryReader.
source_docs = SimpleDirectoryReader('source_docs').load_data()

# Advanced RAG Techniques

## Baseline 'Vanilla' RAG

### Parse source_docs into nodes

In [91]:
from llama_index.node_parser import SimpleNodeParser

# Create a SimpleNodeParser instance with default settings, but with specified chunk overlap and size.
baseline_parser = SimpleNodeParser.from_defaults(
    chunk_overlap=200,
    chunk_size=1024
)

# Use the parser to extract nodes from the documents in 'source_docs'.
baseline_nodes = baseline_parser.get_nodes_from_documents(source_docs)

In [92]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext

# Create a ServiceContext with default settings, including the previously defined language model (llm), embedding model, and node parser.
baseline_context = ServiceContext.from_defaults(llm=llm, embed_model=embedding_model, node_parser=baseline_parser)

# Initialize a VectorStoreIndex with the baseline nodes and the service context.
baseline_index = VectorStoreIndex(baseline_nodes, service_context=baseline_context)

# Persist the baseline index in a directory named "baseline_index".
baseline_index.storage_context.persist(persist_dir="baseline_index")

In [128]:
# Convert the baseline index into a query engine capable of finding the top 3 most similar entries.
baseline_query_engine = baseline_index.as_query_engine(similarity_top_k=3)

# Perform a query with the baseline query engine asking about the influence of camelid genetics on wool quality.
vector_response = baseline_query_engine.query("How do camelid genetics influence wool quality?")

# Retrieve the response from the query.
vector_response.response

'Camelid genetics can influence wool quality. The inheritance of coat colors in alpacas and llamas, which are types of camelids, has been studied. Additionally, major genes affecting alpaca fiber traits have been analyzed. The expression patterns of keratin intermediate filament and keratin associated protein genes in wool follicles have also been investigated. These studies suggest that genetic factors play a role in determining the quality of wool in camelids.'

## Sentence Window Parser

In [99]:
from llama_index.node_parser import SentenceWindowNodeParser

# Initialize a SentenceWindowNodeParser with default settings, including a window size of 6 and specific metadata keys.
sentence_parser = SentenceWindowNodeParser.from_defaults(
    window_size=6,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

# Parse nodes from the documents in 'source_docs' using the sentence parser.
sentence_nodes = sentence_parser.get_nodes_from_documents(source_docs)

# Create a ServiceContext using the sentence parser along with the previously defined language model and embedding model.
sentence_context = ServiceContext.from_defaults(llm=llm, embed_model=embedding_model, node_parser=sentence_parser)

/usr/local/lib/python3.10/dist-packages/llama_index/node_parser/node_utils.py:61: RuntimeWarning: coroutine 'aget_responses' was never awaited
  node = TextNode(


In [100]:
from llama_index import VectorStoreIndex

# Create a VectorStoreIndex with the parsed sentence nodes and the defined service context.
sentence_index = VectorStoreIndex(sentence_nodes, service_context=sentence_context)

# Persist the sentence index in a directory named "sentence_index" for future use.
sentence_index.storage_context.persist(persist_dir="sentence_index")

In [129]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

# Convert the sentence index into a query engine, configuring it to find the top 3 most similar entries.
# It also uses a postprocessor to replace metadata with the 'window' key values.
sentence_query_engine = sentence_index.as_query_engine(
    similarity_top_k=3,
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

# Perform a query using the sentence query engine about the influence of camelid genetics on wool quality.
sentence_response = sentence_query_engine.query("How do camelid genetics influence wool quality?")

# Retrieve the response from the query.
sentence_response.response

'Camelid genetics influence wool quality through various mechanisms. One important aspect is coat color genetics, where llamas and alpacas exhibit a wide range of natural colors and patterns. Llamas, in particular, have greater color variation compared to alpacas. This variation is attributed to the selection process during domestication, where llamas were primarily selected for body size and fiber weight rather than color uniformity or fiber fineness. \n\nAdditionally, the composition and interactions of keratin intermediate filaments (KIFs) and keratin-associated proteins (KAPs) play a crucial role in determining fiber characteristics. Fiber growth in mammals, including camelids, is a cyclical process regulated by genetics, nutrition, and hormones. The proteins that form the fiber are encoded by keratin genes (KRT) and keratin-associated proteins (KRTAP), which are expressed in a highly regulated manner during hair follicle growth.\n\nGenetic selection programs have been implemented 

## Automerging Retrival (Using Hierarchical Nodes)

In [103]:
from llama_index.node_parser import HierarchicalNodeParser

# Initialize a HierarchicalNodeParser with default settings.
hierarchical_parser = HierarchicalNodeParser.from_defaults()

# Parse nodes from the documents in 'source_docs' using the hierarchical parser.
hierarchical_nodes = hierarchical_parser.get_nodes_from_documents(source_docs)

# Create a ServiceContext using the hierarchical parser along with the previously defined language model and embedding model.
hierarchical_context = ServiceContext.from_defaults(llm=llm, embed_model=embedding_model, node_parser=hierarchical_parser)

In [104]:
from llama_index import VectorStoreIndex, StorageContext

# Create a VectorStoreIndex with the parsed hierarchical nodes and the specified service context.
hierarchical_index = VectorStoreIndex(hierarchical_nodes, service_context=hierarchical_context)

# Persist the hierarchical index in a directory named "hierarchical_index" for future use.
hierarchical_index.storage_context.persist(persist_dir="hierarchical_index")

In [130]:
from llama_index.retrievers.auto_merging_retriever import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine

# Initialize an AutoMergingRetriever with the hierarchical index set as the retriever, configured for top 3 similarity matches.
retriever = AutoMergingRetriever(hierarchical_index.as_retriever(similarity_top_k=3), storage_context=hierarchical_index.storage_context, verbose=True)

# Create a RetrieverQueryEngine using the AutoMergingRetriever.
amretriever_query_engine = RetrieverQueryEngine.from_args(retriever)

# Perform a query using the AMRetriever query engine about the influence of camelid genetics on wool quality.
amretriever_response = amretriever_query_engine.query("How do camelid genetics influence wool quality?")

# Retrieve the response from the query.
amretriever_response.response

'Camelid genetics play a significant role in determining wool quality. While there is still much to be understood in this field, recent advancements in genetic understanding have shed light on the genetic mechanisms that regulate economically important fiber traits in South American camelids. Mutations responsible for some monogenic or oligogenic traits have been identified, allowing for molecular testing to assist breeding decisions. Additionally, the development of a 76K SNPs array for the alpaca has facilitated the identification of genes affecting more complex traits through genome-wide association studies. These advancements in genomics and the discovery of genetic variants are expected to contribute to the improvement of wool quality in camelids.'

# Evaluating RAG Performance

### Creating the dataset that will be used for evaluation of RAG methods

In [113]:
import random
from llama_index.evaluation import (DatasetGenerator, QueryResponseDataset)
from llama_index.evaluation import (
    CorrectnessEvaluator,
    SemanticSimilarityEvaluator,
    RelevancyEvaluator,
    FaithfulnessEvaluator
)

# Set the number of nodes to be used for evaluation.
num_nodes_eval = 5

# Randomly select a sample of nodes from baseline_nodes for evaluation.
sample_eval_nodes = random.sample(baseline_nodes, num_nodes_eval)

# Initialize a dataset generator with the sampled nodes, baseline service context, progress display enabled,
# and generating 3 questions per chunk.
dataset_generator = DatasetGenerator(
    sample_eval_nodes,
    service_context=baseline_context,
    show_progress=True,
    num_questions_per_chunk=3,
)

# Asynchronously generate the evaluation dataset from the nodes.
evaluation_dataset = await dataset_generator.agenerate_dataset_from_nodes()

# Initialize evaluators with the baseline service context.
correctness = CorrectnessEvaluator(service_context=baseline_context)
semanticsimilarity = SemanticSimilarityEvaluator(service_context=baseline_context)
relevancy = RelevancyEvaluator(service_context=baseline_context)
faithfulness = FaithfulnessEvaluator(service_context=baseline_context)

<ipython-input-113-8ba2661d47ba>:12: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  dataset_generator = DatasetGenerator(

100%|██████████| 5/5 [00:04<00:00,  1.06it/s]

100%|██████████| 3/3 [00:12<00:00,  4.23s/it]

100%|██████████| 3/3 [00:08<00:00,  2.67s/it]

100%|██████████| 3/3 [00:02<00:00,  1.39it/s]

100%|██████████| 3/3 [00:05<00:00,  1.99s/it]

100%|██████████| 3/3 [00:05<00:00,  1.90s/it]
/usr/local/lib/python3.10/dist-packages/llama_index/evaluation/dataset_generation.py:279: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [116]:
import numpy as np
from llama_index.evaluation import BatchEvalRunner

# Define the maximum number of samples to use for evaluation.
max_samples = 3

# Extract evaluation questions from the evaluation dataset.
evaluation_questions = evaluation_dataset.questions

# Compile expected responses from the question-response pairs in the evaluation dataset.
expected_responses = [response for (question, response) in evaluation_dataset.qr_pairs]

# Create a dictionary mapping evaluation types to their respective evaluator objects.
evaluator_dict = {
    "correctness": correctness,
    "faithfulness": faithfulness,
    "relevancy": relevancy,
    "semanticsimilarity": semanticsimilarity,
}

# Initialize a BatchEvalRunner with the evaluator dictionary, specifying 2 workers and progress display.
batch_eval_runner = BatchEvalRunner(evaluator_dict, workers=2, show_progress=True)

### Evaluate Baseline RAG

In [117]:
from llama_index.evaluation.eval_utils import get_responses, get_results_df

baseline_responses = get_responses(
    evaluation_questions[:max_samples],
    baseline_index.as_query_engine(similarity_top_k=3),
    show_progress=True

)


baseline_evaluation_results = await batch_eval_runner.aevaluate_responses(
    queries=evaluation_questions[:max_samples],
    responses=baseline_responses[:max_samples],
    reference=expected_responses[:max_samples],
)


100%|██████████| 3/3 [00:09<00:00,  3.04s/it]

100%|██████████| 12/12 [00:10<00:00,  1.12it/s]


In [136]:
results_df = get_results_df(
    [baseline_evaluation_results],
    ['Baseline RAG'],
    ["correctness", "relevancy", "faithfulness", "semanticsimilarity"],
)

results_df.rename(columns={'names': 'RAG Method'}, inplace=True)

results_df

,RAG Method,correctness,relevancy,faithfulness,semanticsimilarity
0,Baseline RAG,3.5,1.0,1.0,0.970176


### Evaluating Sentence Window Retrieval

In [120]:
sentence_responses = get_responses(
    evaluation_questions[:max_samples],
    sentence_index.as_query_engine(similarity_top_k=3),
    show_progress=True

)


sentence_evaluation_results = await batch_eval_runner.aevaluate_responses(
    queries=evaluation_questions[:max_samples],
    responses=sentence_responses[:max_samples],
    reference=expected_responses[:max_samples],
)


100%|██████████| 3/3 [00:06<00:00,  2.12s/it]

100%|██████████| 12/12 [00:09<00:00,  1.33it/s]


In [135]:
results_df = get_results_df(
    [sentence_evaluation_results],
    ['Sentence Window Retrieval'],
    ["correctness", "relevancy", "faithfulness", "semanticsimilarity"],
)

results_df.rename(columns={'names': 'RAG Method'}, inplace=True)

results_df

,RAG Method,correctness,relevancy,faithfulness,semanticsimilarity
0,Sentence Window Retrieval,4.666667,1.0,1.0,0.988368


### Evaluating Automerging Retrival

In [122]:
amr_responses = get_responses(
    evaluation_questions[:max_samples],
    amretriever_query_engine,
    show_progress=True

)


amr_evaluation_results = await batch_eval_runner.aevaluate_responses(
    queries=evaluation_questions[:max_samples],
    responses=amr_responses[:max_samples],
    reference=expected_responses[:max_samples],
)


100%|██████████| 3/3 [00:04<00:00,  1.50s/it]

100%|██████████| 12/12 [00:09<00:00,  1.22it/s]


In [123]:
results = get_results_df(
    [amr_evaluation_results],
    ['Automerging Retrieval'],
    ["correctness", "relevancy", "faithfulness", "semanticsimilarity"],
)

results_df.rename(columns={'names': 'RAG Method'}, inplace=True)

results_df

,names,correctness,relevancy,faithfulness,semanticsimilarity
0,Automerging Retrieval,4.166667,1.0,1.0,0.985234


### Summary of Results

In [134]:
results_df = get_results_df(
    [baseline_evaluation_results, sentence_evaluation_results, amr_evaluation_results],
    ['Baseline RAG', 'Sentence Window Retrieval', 'Automerging Retrieval'],
    ["correctness", "relevancy", "faithfulness", "semanticsimilarity"],
)

results_df.rename(columns={'names': 'RAG Method'}, inplace=True)

results_df

,RAG Method,correctness,relevancy,faithfulness,semanticsimilarity
0,Baseline RAG,3.500000,1.0,1.0,0.970176
1,Sentence Window Retrieval,4.666667,1.0,1.0,0.988368
2,Automerging Retrieval,4.166667,1.0,1.0,0.985234
